In [1]:
# Initialize pyterrier
import pyterrier as pt
if not pt.started():
    pt.init()

#create a ColBERT ranking factory based on the pretrained checkpoint
from pyterrier_colbert.ranking import ColBERTFactory
pytcolbert = ColBERTFactory("http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/colbert_model_checkpoint.zip", "./msmarco_index", "msmarco", gpu=True)

# Download and initialize the msmarco dataset
msmarco_ds = pt.get_dataset("msmarco_passage")

# Get list of usable queries that have a corresponding relevant in our limited index
qids = msmarco_ds.get_qrels("dev")

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
/home/gfp4857/miniconda3/envs/project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing ColBERT: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initi

[Jun 14, 12:05:26] #> Loading model checkpoint.
[Jun 14, 12:05:26] #> Loading checkpoint http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/colbert_model_checkpoint.zip


/home/gfp4857/miniconda3/envs/project/lib/python3.11/site-packages/torch/hub.py:665: UserWarning: Falling back to the old format < 1.6. This support will be deprecated in favor of default zipfile format introduced in 1.6. Please redo torch.save() to save it in the new zipfile format.
  warnings.warn('Falling back to the old format < 1.6. This support will be '


[Jun 14, 12:05:36] #> checkpoint['epoch'] = 0
[Jun 14, 12:05:36] #> checkpoint['batch'] = 44500


In [2]:
dense_e2e = pytcolbert.end_to_end()

DEBUG>>> self.args.nprobe=3
[Jun 06, 01:44:42] #> Loading the FAISS index from ./msmarco_index/msmarco/ivfpq.faiss ..
DEBUG>>> faiss_part_range=['ivfpq']
[Jun 06, 01:44:54] #> Building the emb2pid mapping..
[Jun 06, 01:45:32] len(self.emb2pid) = 687758954


RuntimeError: Error in virtual void* faiss::gpu::StandardGpuResourcesImpl::allocMemory(const faiss::gpu::AllocRequest&) at /home/conda/feedstock_root/build_artifacts/faiss-split_1673008980431/work/faiss/gpu/StandardGpuResources.cpp:452: Error: 'err == cudaSuccess' failed: StandardGpuResources: alloc fail type IVFLists dev 0 space Device stream 0x5631b331aa20 size 69132800 bytes (cudaMalloc error out of memory [2])


In [ ]:
from ir_measures import RR

pt.Experiment(
    [dense_e2e],
    msmarco_ds.get_topics("dev"),
    msmarco_ds.get_qrels("dev"),
    eval_metrics=["map", RR@10],
    baseline=0,
    names=["No Pruning"]
)

In [ ]:
del dense_e2e

In [2]:
# Get list of end tokens (e.g., ##s and ##ed)
ids = pytcolbert.unique_token_ids

endtok_ids = []
for id, tok in zip(ids, pytcolbert.token_ids_to_strings(ids)):
    if tok.startswith('##'):
        print(tok, id)
        endtok_ids += [id]

##s 2015
##e 2063
##ing 2075
##t 2102
##a 2050
##er 2121
##r 2099
##d 2094
##y 2100
##l 2140
##n 2078
##c 2278
##ed 2098
##ly 2135
##m 2213
##p 2361
##x 2595
##al 2389
##o 2080
##es 2229
##i 2072
##g 2290
##2 2475
##in 2378
##h 2232
##ic 2594
##0 2692
##te 2618
##us 2271
##on 2239
##k 2243
##ers 2545
##ine 3170
##7 2581
##le 2571
##f 2546
##6 2575
##9 2683
##8 2620
##3 2509
##4 2549
##1 2487
##is 2483
##b 2497
##° 7737
##op 7361
##v 2615
##ia 2401
##um 2819
##ph 8458
##ne 2638
##5 2629
##or 2953
##ry 2854
##co 3597
##ness 2791
##ories 18909
##able 3085
##ive 3512
##est 4355
##ion 3258
##se 3366
##re 2890
##tion 3508
##os 2891
##ct 6593
##u 2226
##lo 4135
##st 3367
##ts 3215
##ma 2863
##ce 3401
##w 2860
##rs 2869
##id 3593
##tic 4588
##ch 2818
##hy 10536
##ns 3619
##en 2368
##ation 3370
##de 3207
##to 3406
##nt 3372
##ity 3012
##an 2319
##as 3022
##ted 3064
##ge 3351
##ra 2527
##ies 3111
##les 4244
##ble 3468
##tes 4570
##th 2705
##rm 10867
##age 4270
##fi 8873
##me 4168
##ter 3334
##sp

In [3]:
# Get list of BERT function tokens 
ids = pytcolbert.unique_token_ids

bert_ids = []
for id, tok in zip(ids, pytcolbert.token_ids_to_strings(ids)):
    if tok.startswith('[') and tok.endswith(']'):
        print(tok, id)
        bert_ids += [id]

[CLS] 101
[SEP] 102
[D] 2
[UNK] 100


In [ ]:
from ir_measures import RR

# Compare the effectiveness of the model before and after only BERT tokens are pruned
dense_e2e = pytcolbert.end_to_end()
dense_e2e_bert_pruned = pytcolbert.end_to_end(bert_ids)

pt.Experiment(
    [dense_e2e, dense_e2e_bert_pruned],
    msmarco_ds.get_topics("dev"),
    msmarco_ds.get_qrels("dev"),
    eval_metrics=["map", RR@10],
    baseline=0,
    names=["No Pruning", "BERT Tokens Pruned"]
)